In [1]:
library(tidyverse)
library(tidytext)
library(topicmodels)
library(tm)

# meter cleaning function
source("src/meter_cleaning.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: NLP

Attaching package: ‘NLP’

The following object is masked from ‘package:ggplot2’:

    annotate



In [2]:
# load dtm
# if you want to replicate this part of the script, don't forget to UZNIP TD_matrix first
wide_dtm = read_tsv("data/TD_matrix_tf_5000.tsv")

Parsed with column specification:
cols(
  .default = col_double(),
  text = col_character()
)
See spec(...) for full column specifications.


Classes ‘spec_tbl_df’, ‘tbl_df’, ‘tbl’ and 'data.frame':	58452 obs. of  4964 variables:
 $ text           : chr  "1311.xhtml_Клушин А.И._Не-титла-знамениты_Я3,4,5,6жжмм" "1060.xhtml_Ржевский А.А._Случилося-двоим_Я1,2,3,4,5,6ж,м" "2861.xhtml_Цветаева М.И._Вы-идущие-мимо-меня_Ан3мж" "154.xhtml_Муравьев М.Н._Успех-твой-первый-возвещая_Я4жм" ...
 $ август         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ аврора         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ автомобиль     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ адам           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ адский         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ азия           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ акация         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ аккорд         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ актер          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ александр      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ алеть          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ алкать         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ аллея          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ алмаз          :

In [3]:
#get doc names and run a meter cleaning function

docs_clean = clean_meter_from_docnames(wide_dtm[,1])

docs_clean[1:10,]

doc
<chr>
1311.xhtml_Клушин А.И._Не-титла-знамениты_NA_NA
1060.xhtml_Ржевский А.А._Случилося-двоим_Явольн_вольн
2861.xhtml_Цветаева М.И._Вы-идущие-мимо-меня_Ан3_мж
154.xhtml_Муравьев М.Н._Успех-твой-первый-возвещая_Я4_жм
753.xhtml_Сумароков А.П._Не-гордитесь-красны-девки_NA_NA
3236.xhtml_Попугаев В.В._Ликуй-Парнас-всплещите-музы_Я4_жм
143.xhtml_Муравьев М.Н._Как-яры-волны-в-море-плещут_Я4_жжм
209.xhtml_Муравьев М.Н._Сей-непорочнейшей-я-кровию-доселе_Я6_жжмм
2601.xhtml_Блок А.А._Ты-была-светла-до-странности_Х4_дм


In [4]:
#prepare matrix
dtm_fin = wide_dtm %>% select(-text) %>% as.matrix()

rm(wide_dtm)
#set names with cleaned meter
rownames(dtm_fin) = docs_clean  %>% pull(doc)

#cast true dtm
dtm_fin = as.DocumentTermMatrix(dtm_fin, weighting = tm::weightTf)

In [18]:
#
save(dtm_fin, file="data/dtm_fin.Rda")